Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50,000 instancias para el entrenamiento, 10,000 para la validación y 10,000 para las pruebas). 

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [2]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True, as_frame=False, parser='auto')

Entrena clasificadores individuales, por ejemplo un clasificador RandomForest, un clasificador SVM y un MLP. 

In [3]:
# Normalizamos los datos
X_mnist = X_mnist / 255.0

# Dividimos el dataset en train, validation y test
X_train, y_train = X_mnist[:50000], y_mnist[:50000]
X_val, y_val = X_mnist[50000:60000], y_mnist[50000:60000]
X_test, y_test = X_mnist[60000:70000], y_mnist[60000:70000]


In [4]:
# Entrenamos un modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=500,
                                       bootstrap=True, 
                                       max_leaf_nodes=16,
                                       n_jobs=-1, 
                                       max_features='sqrt',
                                       oob_score=True,
                                       random_state=0)
random_forest.fit(X_train, y_train)


RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       oob_score=True, random_state=0)

In [5]:
# Score del modelo de Random Forest
random_forest.oob_score_

0.81828

In [6]:
# Validamos el modelo
random_forest.score(X_val, y_val)

0.8395

In [7]:
# Testeamos el modelo
random_forest.score(X_test, y_test)

0.8315

## SVM

In [10]:
# Modelo SVM
svm_clf = SVC(C=0.2, gamma=0.6, random_state=0, tol=2)
svm_clf.fit(X_train, y_train)


SVC(C=0.2, gamma=0.6, random_state=0, tol=2)

A continuación, intenta combinarlos en un ensamble cuyo desempeño supere a cada clasificador individual en el conjunto de validación, utilizando votación por mayoría hard y soft. Una vez que hayas encontrado uno, pruébalo en el conjunto de prueba.

In [11]:
# Validamos el modelo
svm_clf.score(X_val, y_val)

0.9505

In [12]:
# Testeamos el modelo
svm_clf.score(X_test, y_test)

0.9382

## MLP

In [13]:
# Perceptron múltiple
mlp = MLPClassifier(hidden_layer_sizes=(100, ), random_state=0)
mlp.fit(X_train, y_train)

MLPClassifier(random_state=0)

In [14]:
# Validación del modelo
mlp.score(X_val, y_val)

0.9777

In [15]:
# Testeamos el modelo
mlp.score(X_test, y_test)


0.9793

Prueba otros ensambles usando Bagging, Boosting y Stacking (Stacking utiliza validación cruzada por lo que es mejor dividir los datos sólo en entrenamiento y prueba)

In [18]:
# Clasificación por votación
voting_clf = VotingClassifier(
    estimators=[('rf', random_forest),
                ('svm', svm_clf), 
                ('mlp', mlp)]
    )

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_leaf_nodes=16,
                                                     n_estimators=500,
                                                     n_jobs=-1, oob_score=True,
                                                     random_state=0)),
                             ('svm',
                              SVC(C=0.2, gamma=0.6, random_state=0, tol=2)),
                             ('mlp', MLPClassifier(random_state=0))])

In [19]:
voting_clf.score(X_val, y_val)

0.9657

Probamos con soft voting

In [21]:
voting_clf.voting = "soft"
voting_clf.named_estimators["svm"].probability = True
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.979

In [23]:
# AdaBoost
ada_clf = AdaBoostClassifier(svm_clf, n_estimators=100,
                              learning_rate=1, 
                              random_state=0,
                              algorithm='SAMME.R')
ada_clf.fit(X_train, y_train)

: 

In [ ]:
# Bagging
bag_clf = BaggingClassifier(
    svm_clf, 
    n_estimators=500, 
    max_samples=100, 
    max_features = 1.0,
    bootstrap = True,
    oob_score=True,
    n_jobs=-1, 
    random_state=0)

bag_clf.fit(X_train, y_train)

print(bag_clf.oob_score_)